We estimate the joint probability function of the dataset and estimate the entropy of various subsets. Additionally we estimate the conditional probability of game success by looking at the various statistics at our disposal. For our purpose we use the package ks (https://cran.r-project.org/web/packages/ks/index.html), which contains density function estimators.

In [5]:
library(ks)
DATACAPTIONVEC <- c("ID","SEASON","DATE","TEAM1","TEAM2","WON","SCORE","SHOTS","FACEOFF","TAKEAWAY","GIVEAWAY","PIM","HITS","PPG","ATTENDANCE")

Loading required package: KernSmooth
KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009
Loading required package: misc3d
Loading required package: mvtnorm
Loading required package: rgl


In [6]:
nhlDataSum=data.frame()
nhlDataDelta=data.frame()
SeasonVector=c(2010)#,2011,2012,2014,2015,2016)
NumberOfSeasons=length(SeasonVector)
for(season in SeasonVector)
{
  tableName=paste("../dataSetsNHL/dataFileNhl_",season,"_regular_sum.dat",sep="")
  nhlDataS=read.table(tableName)
  colnames(nhlDataS) <- DATACAPTIONVEC
  nhlDataSum<-rbind(nhlDataSum,nhlDataS)
  tableName=paste("../dataSetsNHL/dataFileNhl_",season,"_regular_delta.dat",sep="")
  nhlDataS=read.table(tableName)
  colnames(nhlDataS) <- DATACAPTIONVEC
  nhlDataDelta<-rbind(nhlDataDelta,nhlDataS)
}
colnames(nhlDataSum) <- DATACAPTIONVEC
colnames(nhlDataDelta) <- DATACAPTIONVEC

The input data consists of data with postfix Delta and Sum, with Sum data defined as $X_{\Delta}=X_{h}-X_{a}$ with $X_{h,a} \in X \in \mathbb{D}^{d}$ and $X_{\Sigma}=X_{h}+X_{a}$ with $X_{h,a} \in \mathbb{D}^{d}$ ($\mathbb{D}$ a mixture of binary and natural numbers), d dimensional home and away team statistics vectors. In our case every datasample has 16 entries,with 10 numerical attributes per dataset, giving d=10. The first 6 attributes of each dataset like season, game id and teams who played contain supplementery information and are important only in the preprocessing process. We may write our dataset as "set" $X_{\Delta,Sum} = F ( x_{won},x_{score},x_{shots},x_{faceoff},x_{takeaway},x_{giveaway},x_{pim},x_{hits},x_{ppg},x_{attendance} )$ with F a function of the datasets for home and away teams, as described above. $x_{won}$ takes binary values $x_{won} \in [0,1]$, 0 for loss and 1 for game won. The other attributes take integer values.

The datavectors $X_{Sum},X_{\Delta}$ are now combined to give individual teams statistics. For home games we can simply add the two $X_{team,h}={1 \over 2} \{ X_{Sum} + X_{\Delta} \}$, for the away team stats we have to invert $X_{\Delta}$: $X_{team,a}={1 \over 2} \{ X_{Sum} - X_{\Delta} \}$, and finally add the two: $X_{team}=X_{team,h}+X_{team,a}$ to obtain the overall game stats. We are doing this for all the teams contained in the statistics, which are the following:

In [7]:
teams <- nhlDataDelta$TEAM1
LISTOFTEAMS=unique(teams)
print("we have the following teams")
print(LISTOFTEAMS)

[1] "we have the following teams"
 [1] BOS       TOR       COL       CGY       CAR       PIT       CHI      
 [8] DET       BUF       NJ        NYI       NYR       WSH       ATLANTA  
[15] FLA       CBJ       DAL       STL       EDM       ANA       LA       
[22] VAN       PHI       MIN       TB        NSH       SJ        PHO      
[29] CANADIENS OTT      
30 Levels: ANA ATLANTA BOS BUF CANADIENS CAR CBJ CGY CHI COL DAL DET ... WSH


In [8]:
getTeamGameStatistics <- function(thisTeam)
    {
    matchS<-subset(nhlDataSum,nhlDataSum$TEAM1==thisTeam | nhlDataSum$TEAM2==thisTeam)
    matchH<-subset(nhlDataDelta,nhlDataDelta$TEAM1==thisTeam)
    matchA<-subset(nhlDataDelta,nhlDataDelta$TEAM2==thisTeam)
    
    #invert away data
    matchA$SCORE=-matchA$SCORE
    matchA$SHOTS=-matchA$SHOTS
    matchA$FACEOFF=-matchA$FACEOFF
    matchA$TAKEAWAY=-matchA$TAKEAWAY
    matchA$GIVEAWAY=-matchA$GIVEAWAY
    matchA$PIM=-matchA$PIM
    matchA$HITS=-matchA$HITS
    matchA$PPG=-matchA$PPG

    #add delta data
    matchD<-rbind(matchH,matchA)
    #now order for data then season
    tmp<-matchD[order(matchD$DATE),]
    matchDOrdered<-tmp[order(tmp$SEASON),]

    #compute the NYI values by combining delta and summed data
    teamData=0.5*(matchDOrdered[,sapply(matchDOrdered,is.numeric)]+matchS[,sapply(matchS,is.numeric)])
    return(teamData)
    }

In [9]:
teamsDataList <- list()
for(i in 1:length(LISTOFTEAMS))
{
team=LISTOFTEAMS[i]
teamsDataList[[i]] <- getTeamGameStatistics(team)
}
print(paste("Have retrieved a data list of size",length(teamsDataList),"for ",length(LISTOFTEAMS),"teams"))

[1] "Have retrieved a data list of size 30 for  30 teams"


In the first step we coarse grain the data. One reason for this is to make the computational feat of multidimensional probability function estimation more feasible. The second is that the phase space volume is so vast and data scattered so sparsly, that an agglomeration into more coarse categories may not do harm in reducing the information content. The coarse graining is determined by one parameter only, the number of parts in which every data column is subdivided.

In [10]:
coarseGraining <- function(dataMatrix,parts)
{
  #coarse graining of input data.matrix
  grainedMatrix = matrix(nrow=nrow(dataMatrix),ncol=ncol(dataMatrix))
  centroidMatrix = matrix(nrow=parts,ncol=ncol(dataMatrix))
  for(j in 1:ncol(dataMatrix))
  {
    colVals=dataMatrix[,j]
    coarsedGrainedColData=c()
    maxV=max(colVals)
    minV=min(colVals)
    delta=(maxV-minV)/parts
    centroidV=minV+0:(parts-1)*delta+delta/2
    for(jj in 1:length(colVals))
      {
        #get the index of nearest element of column Values in centroidVector
      actualVal<-colVals[jj]
      coarsedGrainedColData[jj]=centroidV[which.min(abs(centroidV-colVals[jj]))]
    }
    #now add data to matrix
    grainedMatrix[,j] <- coarsedGrainedColData
    centroidMatrix[,j] <- centroidV
  }#next matrix column
  
    #return the coarse grained data matrix and centroid vectors
  return(list("coarseGrainedMatrix"=grainedMatrix,"centroidVectors"=centroidMatrix))
}

In [14]:
nhlDataMatrix=as.matrix(cbind(nhlDataDelta$SHOTS,nhlDataDelta$FACEOFF,nhlDataDelta$HITS,nhlDataDelta$TAKEAWAY,nhlDataDelta$GIVEAWAY,nhlDataDelta$PIM,nhlDataDelta$PPG))
print(paste("We analyze",nrow(matchNYIMatrix),"games characterized by",ncol(matchNYIMatrix),"attributes."))

[1] "We analyze 1223 games characterized by 7 attributes."


In [15]:
returnVal=coarseGraining(nhlDataMatrix,4)
McoarseGrained=returnVal$coarseGrainedMatrix
centroidVectors=returnVal$centroidVectors

In [17]:
  fetchDataColumnsAsMatrix <- function(dataFrame)
  {
    #input dataframe
    #output matrix of certain columns of dataframe, fixed in function
    
    return(as.matrix(cbind(dataFrame$SHOTS,dataFrame$FACEOFF,dataFrame$TAKEAWAY,dataFrame$PIM,dataFrame$PPG)))
    
  }

In [ ]:
Now we write a function to coarse grain data according to a precomputed coarse grain matrix.

In [ ]:
coarseGrainingGrainMatrix <- function(matrixToGrain,centroidVectors)
{
  #categorize input data in matrix grainMatrix according to centroids set in centroidVectors
  #return matrix with values from centroidVectors
  if(is.vector(matrixToGrain))
  {
    matrixToGrain <- as.matrix(matrixToGrain)
    if(ncol(matrixToGrain)<nrow(matrixToGrain))
      {
      matrixToGrain <- t(matrixToGrain)
      }
    #grainedMatrix=as.matrix(matrixToGrain)
  #  grainedMatrix = matrix(nrow=1,ncol=length(matrixToGrain))
  }
  #else
  #{
  grainedMatrix = matrix(nrow=nrow(matrixToGrain),ncol=ncol(matrixToGrain))
  #}
  for(j in 1:ncol(matrixToGrain))
  { 
    centroidV=centroidVectors[,j]
    colVals=matrixToGrain[,j]
    coarsedGrainedColData=c()
    for(jj in 1:length(colVals))
    {
      #get the index of nearest element of column Values in centroidVector
      actualVal<-colVals[jj]
      coarsedGrainedColData[jj] <- centroidV[which.min(abs(centroidV-actualVal))]
    } 
    #now add data to matrix
    grainedMatrix[,j] <- coarsedGrainedColData
  }
  return(grainedMatrix)
}

In the next step we define a simple multivariate probabilit

In [18]:
EvaluesV=c()
  for(team in LISTOFTEAMS)
  {
    matchNYIDH<-subset(nhlDataDelta,nhlDataDelta$TEAM1==team)
    matchDataNYIMatrix<-fetchDataColumnsAsMatrix(matchNYIDH)
    grainedMatchDataNYIMatrix<-coarseGrainingGrainMatrix(matchDataNYIMatrix,centroidVectors)
    Entropy<-getEntropy(McoarseGrained,centroidVectors,grainedMatchDataNYIMatrix)
    #compute average entropy per game
    Entropy<-Entropy/nrow(matchDataNYIMatrix)
    print(Entropy)
    EvaluesV <- c(EvaluesV,Entropy)
  }

[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN
[1] NaN


In [7]:
nhlDataMatrix=as.matrix(cbind(nhlDataDelta$WON,nhlDataDelta$SHOTS))#,nhlDataDelta$FACEOFF))#,nhlDataSum$SHOTS,nhlDataSum$FACEOFF))

In [8]:
estimateProbability <- function(inputVector)
    {
    Pks=kde(nhlDataMatrix,eval.points=inputVector)
    return(Pks$estimate)
    }
#wins,shotsdifference,faceoffdifference,shotssum,faceoffsum
iVec=c(1,5)#,5)#,50,40)
print(estimateProbability(iVec))

[1] 0.22324


In [9]:
getCondProbability <- function(getWin,PMINV,PMAXV)
    {
    #function calculating the conditional probability of win or loss given a parameter value for which to calculate the 
    # probability exceeds the input value
    #input:
    # getWin - 1 if want to now the probability of winning, else 0 (for loosing)
    # parameterSlotIndex - index for the parameter for which to calculate the PDF
    # parameterValue - value of the parameter which is assumed to be exceeded
    P=0
    p1=getWin
    p2min=PMINV[2];p3min=PMINV[2];p4min=PMINV[3];p5min=PMINV[4];
    p2max=PMAXV[2];p3max=PMAXV[2];p4max=PMAXV[3];p5max=PMAXV[4];
    #always integer steps in our statistics
        print(P)
        for(p2 in p2min:p2max){
          #  for(p3 in p3min:p3max){
               # for(p4 in p4min:p4max){
                   # for(p5 in p5min:p5max)
                   #     {
                            iVec=c(p1,p2)#,p3,p4,p5)
                             #result=kde(nhlDataMatrix,eval.points=iVec)
                             #P=P+result$estimate
                            P=P+estimateProbability(iVec)
                            print(P)
                  #  }
              #  }
           # }
        }
    print(P)
    message=paste("We have found a probability of",P," %")
    print(message)
    #return(message)
    }

In [10]:
#the third column goes from 10 shots to max value of 30 shots
PMINV=c(0,-15,10,30,30)
PMAXV=c(1,15,30,50,50)
getCondProbability(1,PMINV,PMAXV)

[1] 0
[1] 0.05386987
[1] 0.1186611
[1] 0.1974263
[1] 0.2926446
[1] 0.4054476
[1] 0.5352806
[1] 0.6800237
[1] 0.8365848
[1] 1.001971
[1] 1.174484
[1] 1.354304
[1] 1.543027
[1] 1.74248
[1] 1.953546
[1] 2.175523
[1] 2.406115
[1] 2.641935
[1] 2.879234
[1] 3.114531
[1] 3.344922
[1] 3.568163
[1] 3.782657
[1] 3.98738
[1] 4.181667
[1] 4.364881
[1] 4.536141
[1] 4.694265
[1] 4.837949
[1] 4.966011
[1] 5.077635
[1] 5.172696
[1] 5.172696
[1] "We have found a probability of 5.17269625108125  %"
